## Hand-written Digit Recognition as a Regression problem
This notebook uses Fireball to creare a regression model. The input to the model is a 28x28 monochrome image from MNIST dataset. The output is the predicted value of the model as a single number. Note that the model does not classify the image. It predicts a floating point value.

This notebook also shows how to subclass a Fireball dataset class. We want the labels in the MNIST dataset to be a floating point value.

## Subclassing the MNIST dataset

In [1]:
import numpy as np
import os

from fireball import Model
from fireball.datasets.mnist import MnistDSet


# Subclassing the original MNIST dataset:
class RegMnistDSet(MnistDSet):
    # ******************************************************************************************************************
    @classmethod
    def postMakeDatasets(cls):
        # This is called at the end of a call to makeDatasets
        cls.numClasses = 0   # Make this a regression dataset
        cls.evalMetricName = 'MSE'
    # ******************************************************************************************************************
    def getBatch(self, batchIndexes):
        return self.samples[batchIndexes], np.float32(self.labels[batchIndexes]) # Return labels as float32

trainDs, testDs = RegMnistDSet.makeDatasets('train,test', batchSize=128)
RegMnistDSet.printDsInfo(trainDs, testDs)


RegMnistDSet Dataset Info:
    Dataset Location ............................... /data/mnist/
    Number of Training Samples ..................... 60000
    Number of Test Samples ......................... 10000
    Sample Shape ................................... (28, 28, 1)


## Create a "Regression LeNet-5" Fireball model and train it on our "RegMnistDSet"
Now let's create a Regression LeNet-5 fireball model using the text string ```layersInfo``` to specify network structure.

In [2]:
# Here we define a "Regression LeNet-5" network which has 2 convolutional layers followed by 3 fully connected
layersInfo = ('IMG_S28_D1,' +                 # The input layer takes a 28x28 image of depth 1 (monochrome)
              'CONV_K5_O6_Ps:ReLU:MP_K2,' +   # Conv, Kernel size 5, 6 out channels, "same" padding, ReLU, Max pool
              'CONV_K5_O16_Pv:ReLU:MP_K2,' +  # Conv, Kernel size 5, 16 out channels, "valid" padding, ReLU, Max pool
              'FC_O120:ReLU,FC_O84:ReLU,FC_O1:ReLU,' +   # 3 fully connected layers
              'REG')                          # Unlike original LeNet-5, the output is just a float32 number

model = Model(name='RegMnistTest',
              layersInfo=layersInfo,
              trainDs=trainDs, testDs=testDs, # Train and test datasets (RegMnistDSet) are given to the model
              numEpochs=10,
              regFactor=0.0001,
              learningRate=(0.001,0.0001),
              optimizer="Adam",
              gpus="0")

model.printLayersInfo()
model.printNetConfig()
model.initSession()
model.train()
model.save("Models/RegMnist.fbm")


Scope            InShape       Comments                 OutShape      Activ.   Post Act.        # of Params
---------------  ------------  -----------------------  ------------  -------  ---------------  -----------
IN_IMG                         Image Size: 28x28x1      28 28 1       None                      0          
L1_CONV          28 28 1       KSP: 5 1 s               14 14 6       ReLU     MP(KSP):2 2 v    156        
L2_CONV          14 14 6       KSP: 5 1 v               5 5 16        ReLU     MP(KSP):2 2 v    2,416      
L3_FC            5 5 16                                 120           ReLU                      48,120     
L4_FC            120                                    84            ReLU                      10,164     
L5_FC            84                                     1             ReLU                      85         
OUT_REG          1                                      1             None                      0          
---------------------------

## Running inference on the trained model
Here we run inference on random samples from the test dataset. Run this several times and see the difference between the actual digit and the predicted value.

In [3]:
i = np.random.randint(testDs.numSamples)
print( "Actual label of the sample no %d in the dataset: %d"%(i, testDs.labels[i]))
print( "Predicted Value: %f"%(model.inferOne(testDs.samples[i])))

Actual label of the sample no 2468 in the dataset: 6
Predicted Value: 5.940730


## Evaluating the model
We can now call the [evaluate](https://interdigitalinc.github.io/Fireball/html/source/model.html#fireball.model.Model.evaluate) function of the Fireball model to evaluate it as a regression problem.


In [4]:
Results = model.evaluate()

  Processed 10000 Sample. (Time: 0.13 Sec.)                              

MSE:  0.298057
RMSE: 0.545946
MAE:  0.261258


## Also look at
[Handwritten Digit recognition (LeNet-5/MNIST)](LeNet5/LeNet5-MNIST.ipynb)

[Reducing number of parameters of LeNet-5 Model](LeNet5-MNIST-Reduce.ipynb)

[Quantizing the LeNet-5 Model](LeNet5-MNIST-Quantize.ipynb)

[Exporting LeNet-5 Model to ONNX](LeNet5-MNIST-ONNX.ipynb)

[Exporting LeNet-5 Model to CoreML](LeNet5-MNIST-CoreML.ipynb)

[Exporting LeNet-5 Model to TensorFlow](LeNet5-MNIST-TF.ipynb)

---

[Fireball Playgrounds](../Contents.ipynb)
